# Deteksi Tepi Plat Nomor Kendaraan


## Import Library


In [ ]:
# %pip install opencv-python matplotlib pillow scikit-image numpy pandas

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
from skimage.measure import shannon_entropy
from skimage.restoration import wiener
import time

## 1. Import Dataset


### Menentukan direktori dataset


In [ ]:
data_dir = './src/dataset/raw data'

### Cek apakah direktori dataset ada


In [ ]:
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Dataset directory '{data_dir}' does not exist. Please check the path.")

### Load dataset


In [ ]:
def load_images_from_directory(directory):
    images = []
    filenames = []
    for filename in os.listdir(directory):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            filepath = os.path.join(directory, filename)
            image = cv2.imread(filepath)
            if image is not None:
                images.append(image)
                filenames.append(filename)
            else:
                print(f"Warning: Unable to load image {filename}.")
    return images, filenames

In [ ]:
# Load images
images, filenames = load_images_from_directory(data_dir)

# Check if images were loaded
if not images:
    raise ValueError("No images were found in the specified directory. Please add some images.")

### Menampilkan dataset


In [ ]:
def display_images(images, filenames, cols=3, title="Gambar"):
    rows = (len(images) + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    for i, ax in enumerate(axes.flat):
        if i < len(images):
            image_rgb = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
            ax.imshow(image_rgb)
            ax.set_title(filenames[i])
            ax.axis('off')
        else:
            ax.axis('off')
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

In [ ]:
display_images(images, filenames, title="Original Images")

## 2. Preprocessing Image


### Folder output


In [ ]:
output_folder = "./src/dataset/preprocessed"
os.makedirs(output_folder, exist_ok=True)

### Konfigurasi kompresi dan reshape


In [ ]:
max_width = 1024  # Resolusi tujuan (width, height)
compression_quality = 100   # Kualitas JPEG (0-100, semakin tinggi semakin baik)

### Fungsi untuk kompresi dan analisis


In [ ]:
analysis_data = []
for filename in os.listdir(data_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(data_dir, filename)
        img = cv2.imread(file_path)

        # Dapatkan dimensi asli
        original_height, original_width = img.shape[:2]

        # Tentukan dimensi target dengan mempertahankan proporsi
        scaling_factor = max_width / original_width
        target_width = int(original_width * scaling_factor)
        target_height = int(original_height * scaling_factor)
        target_size = (target_width, target_height)

        # Resize citra asli sesuai proporsi
        resized_original = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

        # Kompresi citra
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, resized_original, [cv2.IMWRITE_JPEG_QUALITY, compression_quality])

        # Baca kembali citra hasil kompresi
        compressed_img = cv2.imread(output_path)

        # Hitung MSE, PSNR, dan Entropy
        mse_value = mse(resized_original, compressed_img)
        psnr_value = psnr(resized_original, compressed_img)
        entropy_value = shannon_entropy(compressed_img)

        # Simpan hasil analisis
        analysis_data.append({
            "Filename": filename,
            "Original Size (MB)": os.path.getsize(file_path) / (1024 * 1024),
            "Compressed Size (MB)": os.path.getsize(output_path) / (1024 * 1024),
            "MSE": mse_value,
            "PSNR": psnr_value,
            "Entropy": entropy_value,
            "Original Dimensions": f"{original_width}x{original_height}",
            "Resized Dimensions": f"{target_width}x{target_height}"
        })

### Tabel hasil analisis


In [ ]:
df_analysis = pd.DataFrame(analysis_data)
print(df_analysis)

### tampilkan gambar hasil kompresi


In [ ]:
images, filenames = load_images_from_directory(output_folder)
display_images(images, filenames, title="Compressed Images")

### **Analisis Hasil**

1. **MSE dan PSNR**:

   - **MSE**: Nilai MSE berada di kisaran **0.617 hingga 1.183**, yang sangat rendah. Ini menunjukkan bahwa perubahan dari citra asli ke citra hasil kompresi sangat kecil.
   - **PSNR**: Nilai PSNR berkisar antara **47.4 dB hingga 50.2 dB**, yang merupakan kualitas sangat tinggi. Secara visual, perbedaan antara citra asli dan hasil kompresi hampir tidak terlihat.

2. **Entropy**:

   - Nilai entropy berkisar antara **7.29 hingga 7.95**, menunjukkan bahwa informasi dalam citra tetap kaya dan keragaman detail visual tetap terjaga setelah kompresi.

3. **Ukuran File**:
   - Ukuran file berkurang secara signifikan (dari rata-rata 3-4 MB ke kurang dari 1.1 MB), meskipun kualitas JPEG diatur ke **100**.
   - Resolusi citra juga disesuaikan menjadi **1024x1365**, yang mempertahankan proporsi asli.

---

### **Kelebihan**

- **Efisiensi**: Ukuran file yang lebih kecil membuat pemrosesan lebih cepat, terutama saat dataset besar.
- **Kualitas Terjaga**: Nilai PSNR di atas 47 dB memastikan bahwa kompresi tidak merusak kualitas citra secara signifikan.
- **Proporsi Terjaga**: Dimensi gambar mempertahankan rasio asli, yang sangat penting untuk analisis citra.


## 3. image enhancement


### noise reduction (Non-Local Means)


In [ ]:
def reduce_noise(img):
    return cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)

### sharpening menggunakan Laplacian


In [ ]:
def sharpen_image(img):
    kernel = np.array([
        [0, -1, 0],
        [-1, 5, -1],
        [0, -1, 0]
        ])
    return cv2.filter2D(img, -1, kernel)

### mengurangi motion blur menggunakan Wiener Filter


In [ ]:
def reduce_motion_blur(img):
    # Konversi ke grayscale jika gambar memiliki 3 channel
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    kernel = np.ones((5, 5)) / 25  # Kernel untuk motion blur
    img_blur = cv2.filter2D(img, -1, kernel)
    img_float = img_blur.astype(np.float32) / 255.0  # Normalisasi
    restored = wiener(img_float, kernel, 0.1)  # Aplikasikan Wiener filter
    return (restored * 255).astype(np.uint8)  # Konversi kembali ke 8-bit

### mengurangi blur menggunakan Gaussian Blur


In [ ]:
def reduce_blur(img):
    kernel_size = 7  # diubah sesuai kebutuhan
    blurred = cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    return blurred

### meningkatkan kontras menggunakan CLAHE


In [ ]:
def enhance_contrast(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(img)

### Proses peningkatan citra


In [ ]:
# List untuk menyimpan hasil pemrosesan
enhanced_images = []
enhanced_dir = "./src/dataset/enhanced"
os.makedirs(enhanced_dir, exist_ok=True)

for filename in os.listdir(output_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(output_folder, filename)
        img = cv2.imread(file_path)
        
        # Mulai pencatatan waktu
        start_time = time.time()

        # Transformasi ke grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Noise reduction
        denoised_img = reduce_noise(img)

        # Sharpening
        sharpened_img = sharpen_image(denoised_img)

        # Mengurangi motion blur
        motion_blur_reduced = reduce_motion_blur(sharpened_img)

        # Gaussian Blur untuk mengurangi sisa noise
        blurred_img = reduce_blur(motion_blur_reduced)

        # Meningkatkan kontras menggunakan CLAHE
        contrast_enhanced = enhance_contrast(blurred_img)
        
        end_time = time.time()
        processing_duration = end_time - start_time  # Durasi pemrosesan

        # Simpan citra hasil proses
        processed_path = os.path.join(enhanced_dir, f"processed_{filename}")
        cv2.imwrite(processed_path, contrast_enhanced)

        # print(f"{filename} berhasil diproses dan disimpan sebagai {processed_path}.")
        
        # Tambahkan hasil ke list
        enhanced_images.append({
            "Filename": filename,
            "Processed Image": contrast_enhanced
        })

        # print(f"{filename} berhasil diproses dan ditambahkan ke list.")
        
        # Hitung metrik MSE, PSNR, dan Entropy
        mse_value = mse(gray_img, contrast_enhanced)
        psnr_value = psnr(gray_img, contrast_enhanced)
        entropy_value = shannon_entropy(contrast_enhanced)

        # Simpan hasil analisis
        for analysis in analysis_data:
            if analysis["Filename"] == filename:
                analysis["Processing Duration (s)"] = processing_duration
                break

        print(f"{filename} berhasil diproses dalam {processing_duration:.2f} detik.")

In [ ]:
df_analysis = pd.DataFrame(analysis_data)
print(df_analysis)

### Simpan hasil analisis ke file CSV


In [ ]:
# df_analysis.to_csv("./src/analysis/analysis.csv", index=False)

### Fungsi untuk menampilkan perbandingan citra


In [ ]:
def show_comparison(original_images, enhanced_images):
    for original, enhanced in zip(original_images, enhanced_images):
        original_img = cv2.cvtColor(cv2.imread(original["File Path"]), cv2.COLOR_BGR2RGB)  # Konversi ke RGB
        enhanced_img = enhanced["Processed Image"]  # Sudah grayscale

        # Plot perbandingan
        plt.figure(figsize=(10, 5))
        
        # Tampilkan citra asli
        plt.subplot(1, 2, 1)
        plt.imshow(original_img)
        plt.title(f"Original: {original['Filename']}")
        plt.axis("off")

        # Tampilkan citra hasil pemrosesan
        plt.subplot(1, 2, 2)
        plt.imshow(enhanced_img, cmap='gray')
        plt.title(f"Enhanced: {enhanced['Filename']}")
        plt.axis("off")
        
        plt.tight_layout()
        plt.show()

### tampilkan perbandingan


In [ ]:
# Membuat list untuk citra asli
original_images = [{"Filename": filename, "File Path": os.path.join(output_folder, filename)} for filename in os.listdir(output_folder) if filename.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Menampilkan perbandingan
show_comparison(original_images, enhanced_images)

### **Analisis Perbandingan Citra Sebelum dan Setelah Peningkatan**

1. **Peningkatan Kejelasan dan Detail**

   - Citra sebelum peningkatan masih dalam format berwarna, sementara citra setelah peningkatan dikonversi ke grayscale.
   - Setelah peningkatan, plat nomor kendaraan menjadi lebih tajam dan lebih mudah dibaca karena noise yang berkurang serta peningkatan kontras.
   - Bagian kendaraan dan latar belakang juga terlihat lebih terdefinisi, meskipun beberapa area mengalami efek over-enhancement.

2. **Reduksi Noise dan Blur**

   - Noise yang awalnya muncul di bagian gelap dan bayangan pada citra berwarna berhasil dikurangi.
   - Citra yang sebelumnya memiliki blur akibat gerakan atau fokus kamera yang kurang optimal menjadi lebih tajam setelah pemrosesan.
   - Meskipun begitu, pada beberapa area tertentu, efek sharpening dapat menyebabkan munculnya artefak, terutama di tepi objek.

3. **Kontras dan Peningkatan Detail Plat Nomor**
   - Kontras yang lebih tinggi dihasilkan melalui CLAHE, yang membantu dalam memperjelas karakter pada plat nomor.
   - Sebelumnya, beberapa plat nomor tampak buram atau memiliki pantulan cahaya yang mengurangi keterbacaan.
   - Setelah ditingkatkan, karakter pada plat nomor lebih tegas, meskipun ada beberapa area dengan tingkat kecerahan tinggi yang bisa menyebabkan sedikit kehilangan detail.

---

### **Analisis Durasi Peningkatan Setiap Citra**

| Nama Citra  | Durasi Pemrosesan (detik) |
| ----------- | ------------------------- |
| plate-1.jpg | 10.02                     |
| plate-2.jpg | 10.30                     |
| plate-3.jpg | 10.68                     |
| plate-4.jpg | 9.91                      |

#### **Interpretasi Durasi Pemrosesan**

1. **Waktu Pemrosesan Relatif Stabil**

   - Semua gambar diproses dalam rentang waktu 9.91 - 10.68 detik, yang menunjukkan bahwa metode yang digunakan memiliki performa yang konsisten untuk dataset yang serupa.

2. **Perbedaan Durasi Sedikit Dipengaruhi oleh Kompleksitas Citra**

   - Citra **plate-3.jpg** memiliki durasi pemrosesan paling lama (10.68 detik), karena lebih banyak noise atau kompleksitas latar belakang yang lebih tinggi.
   - **plate-4.jpg** memiliki durasi tercepat (9.91 detik), yang disebabkan oleh tingkat noise yang lebih rendah atau struktur yang lebih sederhana pada citra.

3. **Optimasi Durasi Pemrosesan**
   - Jika diperlukan pemrosesan yang lebih cepat, beberapa langkah seperti noise reduction atau Gaussian blur bisa dioptimalkan dengan parameter yang lebih ringan.
   - Penggunaan metode parallel processing atau pemrosesan GPU juga bisa menjadi alternatif untuk mempercepat proses ini dalam skala yang lebih besar.

---

### **Kesimpulan**

- **Citra setelah peningkatan mengalami peningkatan kualitas yang signifikan**, terutama dalam keterbacaan plat nomor kendaraan.
- **Noise dan blur berhasil dikurangi**, membuat kontur lebih jelas dan karakter plat nomor lebih terbaca.
- **Kontras meningkat dengan baik**, meskipun ada beberapa bagian yang mengalami over-enhancement.
- **Durasi pemrosesan cukup stabil**, berkisar di sekitar 10 detik, tetapi tetap bisa dioptimalkan jika dibutuhkan pemrosesan lebih cepat.
